In [ ]:
import socket
import json

# PHD2 server connection details
HOST = 'localhost'
PORT = 4400

def send_request(command):
    """Send a command to the PHD2 server and return the response."""
    with socket.create_connection((HOST, PORT)) as sock:
        sock.sendall(json.dumps(command).encode('utf-8') + b'\n')
        response = sock.recv(4096).decode('utf-8')
        return json.loads(response)

def subscribe_to_events():
    """Subscribe to guide_step events to receive guiding data."""
    command = {
        "method": "set_subscriptions",
        "params": ["guide_step"],  # Subscribe to guide_step event
        "id": 1
    }
    response = send_request(command)
    print("Subscribed to guide_step events:", response)

def listen_for_pixel_changes():
    """Continuously listen for guiding events and extract pixel changes."""
    with socket.create_connection((HOST, PORT)) as sock:
        while True:
            data = sock.recv(4096).decode('utf-8')
            for line in data.split("\n"):
                if line.strip():
                    try:
                        event = json.loads(line)
                        if event.get("Event") == "GuideStep":
                            dx = event.get("dx", 0)  # X pixel change
                            dy = event.get("dy", 0)  # Y pixel change
                            print(f"Pixel Change Detected: ΔX = {dx}, ΔY = {dy}")
                    except json.JSONDecodeError:
                        print("Error decoding JSON:", line)

if __name__ == "__main__":
    print("Connecting to PHD2...")
    subscribe_to_events()
    print("Listening for pixel changes...")
    listen_for_pixel_changes()

Connecting to PHD2...
Subscribed to guide_step events: {'Event': 'Version', 'Timestamp': 1738156473.239, 'Host': 'PEC-E106-D02', 'Inst': 1, 'PHDVersion': '2.6.13', 'PHDSubver': '', 'OverlapSupport': True, 'MsgVersion': 1}
Listening for pixel changes...
Pixel Change Detected: ΔX = 0.391, ΔY = -1.356
Pixel Change Detected: ΔX = 0.783, ΔY = -0.955
Pixel Change Detected: ΔX = 0.493, ΔY = -1.176
Pixel Change Detected: ΔX = 0.908, ΔY = -1.395
Pixel Change Detected: ΔX = 0.938, ΔY = -1.139
Pixel Change Detected: ΔX = 0.999, ΔY = -1.158
Pixel Change Detected: ΔX = 0.831, ΔY = -1.509
Pixel Change Detected: ΔX = 0.992, ΔY = -1.237
Pixel Change Detected: ΔX = 0.576, ΔY = -1.288
Pixel Change Detected: ΔX = 1.02, ΔY = -1.199
Pixel Change Detected: ΔX = 1.034, ΔY = -1.412
Pixel Change Detected: ΔX = 1.147, ΔY = -1.308
Pixel Change Detected: ΔX = 0.97, ΔY = -1.319
Pixel Change Detected: ΔX = 0.837, ΔY = -1.676
Pixel Change Detected: ΔX = 1.092, ΔY = -1.488
Pixel Change Detected: ΔX = 0.904, ΔY = -1.4